
# Coin

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")


import pandas as pd
import datetime as dt

In [2]:
bt = pd.read_csv("data/bitc.csv")
et = pd.read_csv("data/ethereum.csv")
usd = pd.read_csv("data/usdc.csv")

In [3]:
bt.rename(columns={'timestamp':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)
et.rename(columns={'timestamp':'Date','open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)
usd.rename(columns={'timestamp':'Date','open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)

In [4]:
bt['Date'] = pd.to_datetime(bt['Date'])
et['Date'] = pd.to_datetime(et['Date'])
usd['Date'] = pd.to_datetime(usd['Date'])

In [5]:
bt = bt[bt['Date'].between('2021-02-01','2024-05-01')]
et = et[et['Date'].between('2021-02-01','2024-05-01')]
usd = usd[usd['Date'].between('2021-02-01','2024-05-01')]

In [6]:
bt['Date'] = bt['Date'].dt.date
et['Date'] = et['Date'].dt.date
usd['Date'] = usd['Date'].dt.date

In [7]:
bt['Vol/cap'] = bt['Volume'] / bt['Marketcap']
et['Vol/cap'] = et['Volume'] / et['Marketcap']
usd['Vol/cap'] = usd['Volume'] / usd['Marketcap']

In [8]:
bt['HL_spread'] = bt['High'] - bt['Low']
bt['OC_spread'] = bt['Open'] - bt['Close']

et['HL_spread'] = et['High'] - et['Low']
et['OC_spread'] = et['Open'] - et['Close']

usd['HL_spread'] = usd['High'] - usd['Low']
usd['OC_spread'] = usd['Open'] - usd['Close']

In [9]:
bt.describe()

,Open,High,Low,Close,Volume,Marketcap,Vol/cap,HL_spread,OC_spread
count,1185.000000,1185.000000,1185.000000,1185.000000,1.185000e+03,1.185000e+03,1185.000000,1185.000000,1185.000000
mean,37038.727188,37862.437190,36146.777731,37059.018553,3.103575e+10,7.090035e+11,0.047396,1715.659459,-20.291364
std,14188.471516,14581.970237,13721.334271,14200.473631,1.930145e+10,2.709373e+11,0.030982,1482.145621,1356.504573
min,15782.301230,16253.047750,15599.047180,15787.284090,5.331173e+09,3.030000e+11,0.010135,60.824540,-7309.636430
25%,26008.241800,26376.113470,25675.197590,26008.462660,1.871549e+10,5.060000e+11,0.028018,673.804710,-516.966600
50%,35538.608730,36457.796280,34396.477460,35551.958730,2.742502e+10,6.770000e+11,0.039013,1318.859590,5.518310
75%,46707.064450,47881.406230,45544.807510,46707.014880,3.784605e+10,8.830000e+11,0.059268,2332.697400,434.355250
max,73079.373380,73750.073850,71334.092380,73083.501330,3.510000e+11,1.440000e+12,0.406250,12864.619570,7563.999290


In [10]:
et.describe()

,Open,High,Low,Close,Volume,Marketcap,Vol/cap,HL_spread,OC_spread
count,1185.000000,1185.000000,1185.000000,1185.000000,1.185000e+03,1.185000e+03,1185.000000,1185.000000,1185.000000
mean,2294.760298,2356.958227,2226.073538,2295.848105,1.610470e+10,2.740167e+11,0.062053,130.884689,-1.087807
std,865.575019,893.765138,831.726307,864.733948,1.054367e+10,1.014385e+11,0.043000,116.300905,104.781294
min,993.400660,1073.052585,896.109009,993.636805,2.081626e+09,1.204510e+11,0.010320,6.561814,-543.655081
25%,1645.581323,1672.051088,1613.415966,1645.831487,8.355639e+09,1.990000e+11,0.033782,54.472766,-47.778034
50%,1945.033360,2013.755769,1901.361250,1955.389105,1.425761e+10,2.330000e+11,0.047665,100.619766,-1.030730
75%,2937.347167,3017.190395,2860.263668,2940.644827,2.080696e+10,3.522310e+11,0.078848,170.542723,36.689313
max,4810.071533,4891.704698,4718.039256,4812.087614,8.448291e+10,5.690940e+11,0.309442,1485.475556,921.977950


In [11]:
usd.describe()

,Open,High,Low,Close,Volume,Marketcap,Vol/cap,HL_spread,OC_spread
count,1185.000000,1185.000000,1185.000000,1185.000000,1.185000e+03,1.185000e+03,1185.000000,1185.000000,1185.000000
mean,1.000026,1.002057,0.999302,1.000031,4.158645e+09,3.446417e+10,0.133831,0.002755,-0.000005
std,0.001067,0.039513,0.003919,0.000980,2.386230e+09,1.284320e+10,0.081079,0.039742,0.001232
min,0.968300,0.995425,0.877400,0.971500,9.153407e+08,5.830864e+09,0.020701,0.000307,-0.023769
25%,0.999918,1.000363,0.999339,0.999930,2.557259e+09,2.601294e+10,0.081414,0.000769,-0.000138
50%,1.000031,1.000581,0.999611,1.000034,3.596759e+09,3.210414e+10,0.107072,0.001007,-0.000005
75%,1.000153,1.000858,0.999777,1.000156,5.087373e+09,4.439217e+10,0.160665,0.001398,0.000129
max,1.010572,2.349556,1.001303,1.010496,2.668221e+10,5.590056e+10,0.672231,1.351533,0.027970


In [12]:
bt.to_csv('data/bitcoin_price.csv', index=False)
et.to_csv('data/ether_price.csv', index=False)
usd.to_csv('data/usd_price.csv', index=False)

In [13]:
bt = pd.read_csv("data/bitcoin_price.csv")
et = pd.read_csv("data/ether_price.csv")
usd = pd.read_csv("data/usd_price.csv")

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()

In [16]:
bts = bt.set_index('Date',drop=True)
ets = et.set_index('Date',drop=True)
usds = usd.set_index('Date',drop=True)

In [17]:
bitcoin = pd.DataFrame(scaler.fit_transform(bts), columns = bts.columns)
ether = pd.DataFrame(scaler.fit_transform(ets), columns = ets.columns)
usdc = pd.DataFrame(scaler.fit_transform(usds), columns = usds.columns)

In [18]:
bitcoin.index = bt['Date']
ether.index = bt['Date']
usdc.index = bt['Date']

In [19]:
bitcoin.to_csv('data/bitcoin_norm.csv')
ether.to_csv('data/ether_norm.csv')
usdc.to_csv('data/usdc_norm.csv')

# News

In [20]:
news = pd.read_csv("data/news_data_all.csv")

In [21]:
news.drop(news[news['Date']=='Date'].index, axis=0, inplace=True)
news['Date'] = pd.to_datetime(news['Date'])
news = news[news['Date'].between('2021-02-01','2024-05-01')]
news = news.sort_values(by='Date', ascending=False)
news = news.reset_index(drop=True)

In [22]:
news.to_csv('data/news.csv', index=False)

In [23]:
news

,Date,VaderScore,Headline,Summary
0,2024-05-01,0.6369,Whitney Biennial: Winner-Take-All Economy Is R...,This year’s Whitney Biennial shows how America...
1,2024-05-01,0.5574,Bitcoin ($BTC) ETFs Flash Signs of Stress as D...,Prices of some of the biggest spot-Bitcoin exc...
2,2024-05-01,0.6597,AI Hardware Stocks Get Pummeled Even as Big Te...,The good news for makers of semiconductors and...
3,2024-05-01,0.4404,Five Things You Need to Know to Start Your Day...,Good morning. Jerome Powell said the Fed’s nex...
4,2024-05-01,0.1779,Asia Crypto Investing (BTC): Traders Eye Riche...,Everyday traders are betting on digital assets...
...,...,...,...,...
9715,2021-02-04,-0.4019,Bitcoin: One Big Risk When Investing in Crypto...,One of the simplest ways to buy Bitcoin also h...
9716,2021-02-04,-0.1263,Klarna CEO Cautions Executives Promoting Bitco...,Klarna AB Chief Executive Officer Sebastian Si...
9717,2021-02-03,0.0,Futures Debut to Test Ether’s Bitcoin-Beating ...,An eightfold rally in Ether over the past year...
9718,2021-02-01,0.2732,"Musk Says He’s a Bitcoin ‘Supporter,’ Buffetin...",Elon Musk once again sent ripples through the ...


In [24]:
df = pd.DataFrame()

In [25]:
df.index = bt['Date']

In [26]:
df['count'] = 0
df['score_mean'] = 0
df['score_max'] = 0
df['score_min'] = 0
df['score_pos_std'] = 0
df['score_neg_std'] = 0

In [27]:
for i in bt['Date']:
    array = news.loc[news['Date'] == i]['VaderScore'].to_numpy(dtype='float32')
    df['count'][i] = len(array)
    if len(array) != 0:
        df['score_mean'][i] = array.mean()
        df['score_max'][i] = array.max()
        df['score_min'][i] = array.min()
        df['score_pos_std'][i] = array[array > 0].std()
        df['score_neg_std'][i] = array[array < 0].std()

In [28]:
df

,count,score_mean,score_max,score_min,score_pos_std,score_neg_std
Date,,,,,,
2024-04-30,13,0.199823,0.8126,-0.8442,0.200326,0.233978
2024-04-29,12,0.215950,0.8555,-0.1280,0.297651,0.000000
2024-04-28,1,0.361200,0.3612,0.3612,0.000000,NaN
2024-04-27,1,-0.296000,-0.2960,-0.2960,NaN,0.000000
2024-04-26,6,-0.199033,0.0000,-0.7003,NaN,0.103200
...,...,...,...,...,...,...
2021-02-05,1,0.000000,0.0000,0.0000,NaN,NaN
2021-02-04,4,-0.397450,-0.1263,-0.6597,NaN,0.188638
2021-02-03,1,0.000000,0.0000,0.0000,NaN,NaN


In [29]:
df = df.fillna(0)

In [30]:
df.to_csv('data/news_norm.csv',index=True)

In [31]:
df.describe()

,count,score_mean,score_max,score_min,score_pos_std,score_neg_std
count,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000
mean,8.189873,0.070652,0.475620,-0.348383,0.100546,0.071100
std,7.266656,0.210191,0.327425,0.348823,0.100921,0.095302
min,0.000000,-0.872000,-0.872000,-0.975800,0.000000,0.000000
25%,3.000000,-0.021140,0.250000,-0.648600,0.000000,0.000000
50%,7.000000,0.060941,0.570900,-0.401900,0.087182,0.000000
75%,12.000000,0.183293,0.735100,0.000000,0.187195,0.145556
max,69.000000,0.875000,0.980400,0.875000,0.405450,0.379350
